## Analysing daily PQs

In [102]:
import pandas as pd
import requests
from matplotlib import pyplot as plt
from tqdm import tqdm
tqdm.pandas()

In [93]:
!python download.py

tmp folder already exists.
File pqs.csv found with 382655 answered PQs in, last updated 2022-02-28. Looking for new PQs...
Downloaded 0 new PQs, which have been add to the archive.                       
File ua_pqs.csv found with 385097 unanswered PQs in, last updated 2022-02-28. Looking for new PQs...
Downloaded 0 new PQs, which have been add to the archive.                       


In [94]:
# Naughtily, we're importing a database of MP ids & party affiliations from elsewhere... 
active_p = pd.read_csv('active_members.csv')
former_p = pd.read_csv('former_members.csv')

all_p = pd.concat([active_p, former_p])
all_p = all_p[['id', 'nameListAs', 'gender', 'latestPartyabbreviation']]

In [95]:
# Check we get Angie Rayner here
all_p[all_p.id == 4356]

,id,nameListAs,gender,latestPartyabbreviation
437,4356,"Rayner, Angela",F,Lab


In [96]:
id_party_dict = dict(zip(all_p.id, all_p.latestPartyabbreviation))

In [97]:
wpqs = pd.read_csv('tmp/ua_pqs.csv')
wpqs['dateTabled'] = pd.to_datetime(wpqs.dateTabled)
wpqs['heading'] = wpqs.heading.fillna('')
# wpqs = wpqs[['id', 'askingMemberId', 'askingMember', 'house', 'dateTabled', 'questionText', 'answeringBodyName', 'heading']]

# Populate a column with party appreviation in the WPQs database
wpqs['latestPartyabbreviation'] = wpqs.askingMemberId.progress_apply(lambda x: id_party_dict[x] if x in id_party_dict.keys() else 'n/a')

# Make some of the string fields lower case to improve comparability and searchability
wpqs['heading'] = wpqs.heading.progress_apply(lambda x: x.lower())
wpqs['questionText'] = wpqs.questionText.progress_apply(lambda x: x.lower())

# Sometime the heading is a generic topic, other times it's specified by a ":" symbol. We'll extract this into a 'topic' column.
wpqs['topic'] = wpqs.heading.progress_apply(lambda x: x.split(':')[0])

100%|██████████| 385097/385097 [00:00<00:00, 1292154.57it/s]


In [123]:
wpqs.head()

,id,askingMemberId,askingMember,house,memberHasInterest,dateTabled,dateForAnswer,uin,questionText,answeringBodyId,answeringBodyName,heading,latestPartyabbreviation,topic
0,58429,437,NaN,Commons,False,2014-06-05,2014-06-09T00:00:00,199151,"to ask mr chancellor of the exchequer, how muc...",14,HM Treasury,working tax credit: bolton,Lab,working tax credit
1,57913,4031,NaN,Commons,False,2014-06-04,2014-06-09T00:00:00,198956,"to ask mr chancellor of the exchequer, what th...",14,HM Treasury,welfare tax credits,Lab,welfare tax credits
2,61038,4031,NaN,Commons,False,2014-06-10,2014-06-16T00:00:00,199850,"to ask mr chancellor of the exchequer, what th...",14,HM Treasury,welfare tax credits,Lab,welfare tax credits
3,64002,163,NaN,Commons,False,2014-06-25,2014-06-30T00:00:00,202509,"to ask mr chancellor of the exchequer, how man...",14,HM Treasury,welfare tax credits: self-employed,Lab,welfare tax credits
4,64577,385,NaN,Commons,False,2014-06-27,2014-07-01T00:00:00,202934,"to ask mr chancellor of the exchequer, how man...",14,HM Treasury,welfare tax credits: york,Lab,welfare tax credits


In [136]:
wpqs[wpqs.topic.str.contains('electric' 'vehicles')]

,id,askingMemberId,askingMember,house,memberHasInterest,dateTabled,dateForAnswer,uin,questionText,answeringBodyId,answeringBodyName,heading,latestPartyabbreviation,topic
11815,91095,249,NaN,Commons,False,2014-09-08,2014-09-10T00:00:00,208588,"to ask the secretary of state for transport, w...",27,Department for Transport,electric vehicles: greater london,Con,electric vehicles
18980,156425,172,NaN,Commons,False,2014-11-18,2014-11-20T00:00:00,214862,"to ask the secretary of state for transport, w...",27,Department for Transport,electric vehicles: buses,Lab,electric vehicles
35952,226071,301,NaN,Commons,False,2015-03-09,2015-03-12T00:00:00,226665,"to ask the secretary of state for transport, w...",27,Department for Transport,electric vehicles: bicycles,Lab,electric vehicles
40813,353889,146,NaN,Commons,False,2015-06-12,2015-06-16T00:00:00,2332,"to ask the secretary of state for transport, h...",27,Department for Transport,electric vehicles: charging points,Lab,electric vehicles
40837,353891,146,NaN,Commons,False,2015-06-12,2015-06-16T00:00:00,2358,"to ask the secretary of state for transport, w...",27,Department for Transport,electric vehicles: charging points,Lab,electric vehicles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382866,1419525,188,NaN,Commons,False,2022-02-03,2022-02-09T00:00:00,117736,"to ask the secretary of state for transport, w...",27,Department for Transport,electric vehicles: charging points,LD,electric vehicles
382867,1419524,188,NaN,Commons,False,2022-02-03,2022-02-09T00:00:00,117735,"to ask the secretary of state for transport, w...",27,Department for Transport,electric vehicles: disability,LD,electric vehicles
383004,1418606,4131,NaN,Commons,False,2022-02-01,2022-02-03T00:00:00,115913,"to ask the secretary of state for transport, w...",27,Department for Transport,electric vehicles: charging points,DUP,electric vehicles
383127,1419184,4571,NaN,Commons,False,2022-02-02,2022-02-07T00:00:00,117009,"to ask the secretary of state for transport, h...",27,Department for Transport,electric vehicles: charging points,Lab,electric vehicles
